# WQP Data Retrieval

WQP Data Retrieval provides an easy way to obtain data points corresponding to a lake from the water quality data portal (https://www.waterqualitydata.us/portal/). <br>
Given a set of site IDs <i>X</i> that correspond to a single lake, a set of columns of interest <i>Y</i>, and a set of characteristics of interest <i>Z</i>, this script will query WQP (water quality portal) to obtain data for all site IDs in <i>X</i>, then pull out only the columns in <i>Y</i>, then filter the rows based on the characteristics in <i>Z</i>. This will produce a <i>pandas DataFrame</i> with data points. 

## Function definitions

In this section of notebook, the functions used to obtain data from WQP are defined.<br>
1. Function to write contents of csv into list<br>
2. Function to fetch data from WQP, given site IDs <i>X</i><br>
3. Function to retain columns of interest <i>Y</i> in DataFrame<br>
4. Function to filter rows based on characteristics of interest <i>Z</i>
5. Function that performs steps (2),(3) and (4) and returns the data of interest

In [51]:
import pandas as pd
from pywqp import pywqp_client
client_instance = pywqp_client.RESTClient()

### Function to write contents of csv into list

This function writes contents of csv file into a list. It takes path to csv file as input argument and writes contents of the csv file into a list.

In [52]:
def csv_to_list(file_path):
    """
        Writes content of csv into list
        Args:
            file_path (str): String
        Returns:
            contents_list (list)
    """
    df = pd.read_csv(file_path,header=None) 
    contents_list = df[0].tolist()
    return contents_list

### Function to fetch data from WQP, given site IDs X
This function fetches data from the WQP for a given set of site IDs corresponding to a lake using a web service client. It takes path to csv file containing site IDs as input argument, makes a service call and writes the response data into a pandas DataFrame.

In [53]:
def web_service_call(dict_place,file_path_site_ids):
    """
        Retrieving data from water quality portal using web service client
        Args:
            dict_place (dict): Input dictionary ,file_path_site_ids (str): Input String 
        Returns:
            result_df (pandas DataFrame)
    """
    # Get the site ids
    list_site_ids = csv_to_list(file_path_site_ids)
    site_ids = ';'.join(list_site_ids)
    
    # Preparing the input payload
    verb = 'get'
    host_url = 'http://waterqualitydata.us'
    resource_label = 'result'
    params = {'countrycode': dict_place['country_code'], 'statecode': dict_place['state_code'], 'countycode': dict_place['county_code'],'siteid':site_ids}
    #params = {'countrycode': '', 'statecode': '', 'countycode': '','siteid':site_ids}
    equivalent_url = client_instance.create_rest_url(host_url, resource_label, params, mime_type='text/csv')
    
    # Make the web-service call and get the response into a DataFrame
    result = client_instance.request_wqp_data(verb, host_url, resource_label, params, mime_type='text/csv')
    result_df = client_instance.response_as_pandas_dataframe(result)
    return result_df

### Function to retain columns of interest Y in DataFrame
This function retains only the columns of interest in the given DataFrame. It takes as input DataFrame and path to csv file containing column names and retains only the given columns in DataFrame.

In [54]:
def format_columns_df(df,file_path_columns):
    """
        Retaining required columns in DataFrame
        Args:
            df: Input pandas dataframe, file_path_columns (str): Input String
        Returns:
            df_drop_cols (pandas DataFrame)
    """
    # Get list of column names from given file path
    columns = csv_to_list(file_path_columns)
    
    # Get the existing columns in DataFrame
    columns_init = df.columns.tolist()
    
    # Check for existence of columns in DataFrame
    if set(columns).issubset(columns_init):
        df_drop_cols = df[columns]
        return df_drop_cols
    else:
        print("Invalid column names present")

### Function to filter rows based on characteristics of interest Z
This function filters rows in the DataFrame based on given characteristics. It takes as input DataFrame and path to csv file containing <i>CharacteristicName</i> values and filters the DataFrame on the given values.

In [55]:
def filter_df(df,file_path_characteristics):
    """
        Filtering column 'CharacteristicName' of DataFrame
        Args:
            df: Input pandas dataframe, file_path_characteristics (str): Input String
        Returns:
            df_filtered (pandas DataFrame)
    """
    # Get list of column names from given file path
    characteristics = csv_to_list(file_path_characteristics)
    df_filtered = df[df['CharacteristicName'].isin(characteristics)]
    return df_filtered
    

### Function that performs steps (2),(3) and (4) and returns the data of interest
This function queries WQP (water quality portal) to obtain data for all site IDs, then pulls out only the columns  of interest, then filters the rows based on the characteristics of interest and returns a DataFrame containing the data of interest.
It takes as input path to csv file containing site IDs, path to csv file containing columns of interest, path to csv file containing characteristics of interest and returns the desired data points.

In [56]:
def retrieve_data(dict_place,file_path_site_ids,file_path_columns,file_path_characteristics):
    """
        Retrieves data from WQP for desired lake
        Args:
            dict_place (dict): Input dictionary, file_path_site_ids (str): Input pandas dataframe, 
            file_path_columns (str): Input String, file_path_characteristics (str): Input String 
        Returns:
            df_filtered_chars (pandas DataFrame)
    """
    #Fetch data points in a DataFrame for the given site IDs.
    df_with_site_ids = web_service_call(dict_place,file_path_site_ids)

    #Retain the required columns in the DataFrame.
    df_drop_cols = format_columns_df(df_with_site_ids,file_path_columns)

    #Filter the DataFrame based on required characteristics.
    df_filtered_chars = filter_df(df_drop_cols,file_path_characteristics)
    return df_filtered_chars

---------------------------

## Data Retrieval for Lake Mendota
In this section, we use the functions defined above to retrieve data points from WQP for Lake Mendota.<br>
Given to us are the below paths:
1. Dictionary of country code, state code and county code
2. Path to csv file containing site IDs corresponding to Lake Mendota
3. Path to csv file containing columns of interest
4. Path to csv file containing characteristics of interest

We obtain a DataFrame containing the data points for Lake Mendota.<br>

In [57]:
# Arguments for the function retrieve_data
dict_place = {}
dict_place['country_code'] = 'US'
dict_place['state_code'] = 'US:55'
dict_place['county_code'] = 'US:55:025'

file_path_site_ids = '../data/site_ids.csv'
file_path_columns = '../data/columns.csv'
file_path_characteristics = '../data/characteristics.csv'

#Function call to retrive data for Lake Mendota
df = retrieve_data(dict_place,file_path_site_ids,file_path_columns,file_path_characteristics)

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:11: DtypeWarning: Columns (5,24,25,30,41,47,49,50,58,60) have mixed types. Specify dtype option on import or set low_memory=False.


In [58]:
df

,ActivityIdentifier,ActivityStartDate,ActivityStartTime/Time,ActivityStartTime/TimeZoneCode,ActivityEndDate,ActivityEndTime/Time,ActivityEndTime/TimeZoneCode,ActivityDepthHeightMeasure/MeasureValue,ActivityDepthHeightMeasure/MeasureUnitCode,ActivityDepthAltitudeReferencePointText,MonitoringLocationIdentifier,SampleCollectionMethod/MethodIdentifier,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,ResultAnalyticalMethod/MethodIdentifier,ResultAnalyticalMethod/MethodIdentifierContext,ResultAnalyticalMethod/MethodName,ProviderName
1,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Oxygen,Dissolved,7.6,mg/l,NaN,NaN,NaN,NWIS
2,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,pH,Total,8.6,std units,NaN,NaN,NaN,NWIS
4,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,"Temperature, water",NaN,20.7,deg C,NaN,NaN,NaN,NWIS
6,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Ammonia and ammonium,Dissolved,0.103,mg/l NH4,ALGOR,USGS,Computation by NWIS algorithm,NWIS
7,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Ammonia and ammonium,Dissolved,0.080,mg/l as N,CL035,USGS,"Ammonia, wf, ASF salicylate",NWIS
8,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Inorganic nitrogen (nitrate and nitrite),Dissolved,NaN,NaN,CL045,USGS,"NO2+NO3, wf, Cd reduction ASF",NWIS
9,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Nitrite,Dissolved,NaN,NaN,CL040,USGS,"Nitrite, wf, ASF diazotization",NWIS
11,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Hydrogen ion,Total,NaN,NaN,ALGOR,USGS,Computation by NWIS algorithm,NWIS
13,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Nitrate,Dissolved,NaN,NaN,ALGOR,USGS,Computation by NWIS algorithm,NWIS
14,nwiswi.01.99204088,1992-07-23,14:10:00,CDT,NaN,NaN,NaN,1.5,feet,NaN,USGS-05427968,USGS,Nitrate,Dissolved,NaN,NaN,ALGOR,USGS,Computation by NWIS algorithm,NWIS


We can write the obtained DataFrame containing the data points of Lake Mendota into a csv file of desired path.<br>
Now we can see the data for Lake Mendota in <i>data/lake_mendota_data.csv</i>

In [59]:
df.to_csv('../data/lake_mendota_data.csv',index=False)